<a href="https://colab.research.google.com/github/Lamblador/IR_expert_system/blob/main/Koshelev_IR_expert_system_2_First_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR spectral analysis of organic and coordination compounds via machine learning and neural networks
By: Koshelev Daniil 

Based on some code form: https://github.com/rayguna/IR-Spectra-Analyzer
Предыдущая часть: https://github.com/Lamblador/IR_expert_system/blob/main/Koshelev_IR_expert_system_1_Dataset_Preparation.ipynb

-Work in progress (last updated:10/03/2023)-

B. Цель данного блокнота

Целью этого блокнота является обучение примитивной модели распознавания всего лишь 15 классов (функциональных групп) в спектре из 1800 точек в диапазоне 500-4100 [1/cm] 

C. План реализации

1. Импортировать табличные данные разметки классов по CAS и табличные данные уже обработанных спектров нужного диапазона.
2. Создать объект класса Dataset
3. Реализовать код для обучения модели.
4. Реализовать саму модель.
5. Обучить модель.
6. Оценить точность модели в мультиклассовом анализе.

NOTES
1. Импортированы табличные данные.

Клонирование данных с GitHub. На сайте выложены как исходные списки веществ, так и скачанные спектры соединений, которые были запарсены кодом ниже.

In [1]:
!git clone https://github.com/Lamblador/IR_expert_system.git

Cloning into 'IR_expert_system'...
remote: Enumerating objects: 16753, done.
remote: Counting objects: 100% (1910/1910), done.
remote: Compressing objects: 100% (1856/1856), done.
remote: Total 16753 (delta 64), reused 1894 (delta 53), pack-reused 14843
Receiving objects: 100% (16753/16753), 244.29 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (424/424), done.
Updating files: 100% (16623/16623), done.


In [5]:
# Установка нужных пакетов
from IPython.core.display import clear_output
!pip install --upgrade jcamp
clear_output()

In [3]:
# Вводятся нужные импорты
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os
import numpy as np
import torch
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

# Импорт табличных данных

In [4]:
labels_df = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_inchi_group_list.csv')
labels_df = labels_df.set_index("CAS")
labels_df

,Name,Formula,Mw,Elements,InChi,SMILES,alkane,methyl,alkene,alkyne,...,aromatics,alkyl halides,esters,ketones,aldehydes,carboxylic acids,ether,acyl halides,amides,nitro
CAS,,,,,,,,,,,,,,,,,,,,,
100-00-5,"Benzene, 1-chloro-4-nitro-",C6H4ClNO2,156.993056,"['C', 'H', 'Cl', 'N', 'O']",InChI=1/C6H4ClNO2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H,[O-][N+](=O)c1ccc(Cl)cc1,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100-01-6,p-Nitroaniline,C6H6N2O2,138.042927,"['C', 'H', 'N', 'O']",InChI=1/C6H6N2O2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H...,Nc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100-02-7,"Phenol, 4-nitro-",C6H5NO3,139.026943,"['C', 'H', 'N', 'O']","InChI=1/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",Oc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000-49-3,N-butyltrimethylsilane,C7H18Si,130.117777,"['C', 'H', 'Si']","InChI=1/C7H18Si/c1-5-6-7-8(2,3)4/h5-7H2,1-4H3",CCCC[Si](C)(C)C,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000-50-6,N-Butyldimethylchlorosilane,C6H15ClSi,150.063155,"['C', 'H', 'Cl', 'Si']","InChI=1/C6H15ClSi/c1-4-5-6-8(2,3)7/h4-6H2,1-3H3",CCCC[Si](C)(C)Cl,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99-97-8,"Benzenamine, N,N,4-trimethyl-",C9H13N,135.104799,"['C', 'H', 'N']","InChI=1/C9H13N/c1-8-4-6-9(7-5-8)10(2)3/h4-7H,1...",CN(C)c1ccc(C)cc1,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999-78-0,"2-Pentyne, 4,4-dimethyl-",C7H12,96.093900,"['C', 'H']","InChI=1/C7H12/c1-5-6-7(2,3)4/h1-4H3",CC#CC(C)(C)C,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999-81-5,"Ammonium chloride, 2-chloroethyltrimethyl-",C5H13ClN.Cl,157.042505,"['C', 'H', 'Cl', 'N', 'Cl']","InChI=1/C5H13ClN.ClH/c1-7(2,3)5-4-6;/h4-5H2,1-...",[Cl-].C[N+](C)(C)CCCl,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Импорт данных и обработка спектров

In [6]:
from jcamp import jcamp_readfile
nist_ir_spectra_list_with_group = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_inchi_group_list.csv')
# Create an empty list to store spectra data
ref_spectra = []
# choose current directory
curr_directory = os.getcwd()
os.chdir(curr_directory)  # set to current directory
files = glob.glob('/content/IR_expert_system/downloaded_jcamp/ir/*')  # look for files without extension. Returns a list.
# Try to read the file. If can't be read, skip it.
dataset_dic = {}
for i, file in enumerate(files):
    
    #print(file)
    try:
        # read jcamp file using the jcamp class and store data into the
        # ref_spectra list
        sample = jcamp_readfile(file)
        #Forming data into the types
        sample['x'] = sample['x'].astype(np.float32)
        sample['y'] = sample['y'].astype(np.float32)
        sample['firstx'] = np.float32(sample['x'][0])
        #print(type(sample['firstx']))
        sample['deltax'] = ((np.float32(sample['firstx'])-np.float32(sample['lastx']))/np.float32(sample['npoints']))
        sample['npoints'] = int(sample['npoints'])    
        # uniformize data, #1:
        # check yunits: If yunits is in transmission, change it to absorbance.
        # Later create a function to uniformize units
        if sample['yunits'] == "ABSORBANCE":
            pass 

        elif sample['yunits'] == "TRANSMISSION" or sample['yunits'] == "TRANSMITTANCE":
            
            sample['y'] = 1 - sample['y']
            sample['yunits'] = "ABSORBANCE"
            
            #normalize
           
        # check xunits: if in microns, change to 1/cm

        if sample['xunits'] == "MICROMETERS":
            sample['x'] = 10000 / sample['x']
            sample['xunits'] = '1/CM'
            sample['firstx'] = np.float32(sample['x'][0])
            sample['lastx'] = np.float32(sample['x'][int(sample['npoints'])-1])
            sample['deltax'] = ((np.float32(sample['firstx'])-np.float32(sample['lastx']))/np.float32(sample['npoints']))
            #if sample['deltax'] == np.nan: print(float(sample['firstx'])-float(sample['lastx'])), float(sample['npoints'])
            sample['maxx'] =  np.float32(max(sample['x']))
            sample['minx'] =  np.float32(min(sample['x']))
        # check deltax: if Nan, change to (maxx-minx)/npoints
        #if sample['deltax'] == np.nan: sample['deltax'] = ((float(sample['firstx'])-float(sample['lastx']))/float(sample['npoints']))

        # !Still need to uniformize x-axis range and uniformize x-axis intervals.
        #Flitting the x array if it is not in min_to_max order
        if sample['x'][0] > sample['x'][3]: 
           #print(sample['x'])
           sample['x'] = np.flip(sample['x'])
           #print(sample['x'])
           sample['y'] = np.flip(sample['y'])
           sample['firstx'] = np.float32(sample['x'][0])
           sample['lastx'] = np.float32(sample['x'][int(sample['npoints'])-1])
           
        
        #Saving data to dictionary under cas num
        dataset_dic[sample['cas registry no']] = sample
        #print(sample['cas registry no'])
        # Nan check
        if np.isnan(sample['y']).any(): print("Reading jcamp problem! Nan observed in Y")
                

    except:
         print("File can't be read, please check:", file)
         continue

#print(dataset_dic)

File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/4215-80-9
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/1503-53-3
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/103-01-5
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/57338-87-1
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/606-12-2
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/22539-65-7
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/22303-30-6
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/90920-28-8
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/586-75-4
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/54932-77-3
File can't be read, please check: /content/IR_expert_system/downloaded_jcamp/ir/4097-48-7
File can

In [7]:
dataset_df = pd.DataFrame(dataset_dic)
dataset_df = dataset_df.transpose()
dataset_df[['x', 'y', 'minx', 'maxx', 'npoints']]

,x,y,minx,maxx,npoints
4935-80-2,"[461.0, 462.0, 463.0, 464.0, 465.0, 466.0, 467...","[0.30980003, 0.30980003, 0.3099, 0.31019998, 0...",461,3798,3338
56-81-5,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[0.0011424, 0.0016443, 0.0010185, 0.0010815, 0...",550,3846,825
1615-70-9,"[625.52386, 625.7912, 626.0587, 626.32654, 626...","[0.41900003, 0.41900003, 0.41900003, 0.417, 0....",625.523865,4950.495117,2049
1078-71-3,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[8.32e-05, 0.000192, 0.0002464, 0.0003424, 0.0...",550,3846,825
4727-17-7,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[0.0001898, 8.76e-05, 5.84e-05, 4.38e-05, 2.92...",550,3846,825
...,...,...,...,...,...
1517-22-2,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[0.002968, 0.0069352, 0.0064488, 0.0050144, 0....",550,3846,825
137-05-3,"[624.7501, 625.0176, 625.2853, 625.5532, 625.8...","[0.19700003, 0.195, 0.195, 0.195, 0.194, 0.194...",624.750122,4933.399414,2045
60263-54-9,"[626.4369, 626.70483, 626.973, 627.2414, 627.5...","[0.14099997, 0.14099997, 0.13999999, 0.1380000...",626.43689,5000.0,2045
2437-33-4,"[625.7979, 626.0643, 626.331, 626.59717, 626.8...","[0.453, 0.44300002, 0.435, 0.42799997, 0.41900...",625.797913,4835.589844,2043


In [9]:
#Добавление к краям с исходным шагом крайнего значения y.
def padding_ir_below(x, y, low_border):
  '''
    x, y - array-like data
    low_border - border until which padding will continue 

    return padded x and y as lists
  '''
  x, y = list(x), list(y)
  deltax = x[1]-x[0]
  next_x = x[0]-deltax
  while next_x > low_border:
    x.insert(0, next_x)
    y.insert(0, y[0])
    next_x = x[0]-deltax
  #Последняя итерация чтобы выйти за границу интерполирования
  x.insert(0, next_x)
  y.insert(0, y[0])
  next_x = x[0]-deltax
  return x, y

def padding_ir_above(x, y, high_border):
  '''
    x, y - array-like data
    high_border - border until which padding will continue 


    return padded x and y as lists
  '''
  x, y = list(x), list(y)
  deltax = x[1]-x[0]
  next_x = x[-1]+deltax
  while next_x < high_border:
    x.append(next_x)
    y.append(y[-1])
    next_x = x[-1]+deltax
  #Последняя итерация чтобы выйти за границу интерполирования
  x.append(next_x)
  y.append(y[-1])
  next_x = x[-1]+deltax
  return x, y

from scipy import interpolate

def interpolate_in_borders_with_delta(x, y, borders = [500, 4100], deltax = 2):
  '''
  x, y - array like data
  borders - list of [low_border, high border]
  deltax - increment of new data

  returns new x, y data as lists with increment = deltax
  '''
  x, y = list(x), list(y)
  f = interpolate.interp1d(x, y)
  #print(f(x[0]))
  x_new = []
  for i in range(borders[0], borders[1], deltax):
    x_new.append(i)
  x_new = np.array(x_new)
  y_new = f(x_new)
  
  return x_new, y_new



In [10]:
x_list, y_list = [], []
errors = 0
for i in tqdm(range(len(dataset_df[['x']]))):
  try:
      x, y = dataset_df[['x']].iloc[i].tolist(), dataset_df[['y']].iloc[i].tolist()
      x1, y1 = padding_ir_below(x[0], y[0], 500.0)

      x2, y2 = padding_ir_above(x1, y1, 4100.0)
  
      x3, y3 = interpolate_in_borders_with_delta(x2, y2)
      x_list.append(x3)
      y_list.append(y3)
  except ValueError:
      #print(f'0: {len(x)}, {len(y)}, 1: {len(x1)}, {len(y1)}, 2: {len(x2)}, {len(y2)}')
      errors +=1
      x_list.append(np.nan)
      y_list.append(np.nan) 
print(f'Found {errors} errors')

100%|██████████| 14346/14346 [01:18<00:00, 183.70it/s]

Found 38 errors


In [11]:
x_serie = pd.Series(x_list)
y_serie = pd.Series(y_list)

In [12]:
df = pd.DataFrame(x_serie, columns = ['x1'])
df['y1'] = y_serie
df

,x1,y1
0,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.33560001850128174, 0.32169997692108154, 0.2..."
1,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.001142399967648089, 0.001142399967648089, 0..."
2,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.4190000295639038, 0.4190000295639038, 0.419..."
3,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[8.320000051753595e-05, 8.320000051753595e-05,..."
4,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.00018980000459123403, 0.0001898000045912340..."
...,...,...
14341,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.002968000015243888, 0.002968000015243888, 0..."
14342,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.19700002670288086, 0.19700002670288086, 0.1..."
14343,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.14099997282028198, 0.14099997282028198, 0.1..."
14344,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.453000009059906, 0.453000009059906, 0.45300..."


In [41]:
dataset_df = dataset_df.reset_index()
dataset_df1 = dataset_df.join(df, lsuffix='x')
dataset_df1[['x', 'x1', 'y', 'y1']]

,x,x1,y,y1
0,"[461.0, 462.0, 463.0, 464.0, 465.0, 466.0, 467...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.30980003, 0.30980003, 0.3099, 0.31019998, 0...","[0.33560001850128174, 0.32169997692108154, 0.2..."
1,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0011424, 0.0016443, 0.0010185, 0.0010815, 0...","[0.001142399967648089, 0.001142399967648089, 0..."
2,"[625.52386, 625.7912, 626.0587, 626.32654, 626...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.41900003, 0.41900003, 0.41900003, 0.417, 0....","[0.4190000295639038, 0.4190000295639038, 0.419..."
3,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[8.32e-05, 0.000192, 0.0002464, 0.0003424, 0.0...","[8.320000051753595e-05, 8.320000051753595e-05,..."
4,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0001898, 8.76e-05, 5.84e-05, 4.38e-05, 2.92...","[0.00018980000459123403, 0.0001898000045912340..."
...,...,...,...,...
14341,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.002968, 0.0069352, 0.0064488, 0.0050144, 0....","[0.002968000015243888, 0.002968000015243888, 0..."
14342,"[624.7501, 625.0176, 625.2853, 625.5532, 625.8...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.19700003, 0.195, 0.195, 0.195, 0.194, 0.194...","[0.19700002670288086, 0.19700002670288086, 0.1..."
14343,"[626.4369, 626.70483, 626.973, 627.2414, 627.5...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.14099997, 0.14099997, 0.13999999, 0.1380000...","[0.14099997282028198, 0.14099997282028198, 0.1..."
14344,"[625.7979, 626.0643, 626.331, 626.59717, 626.8...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.453, 0.44300002, 0.435, 0.42799997, 0.41900...","[0.453000009059906, 0.453000009059906, 0.45300..."


In [42]:
dataset_df1['y1'][0]

array([0.33560002, 0.32169998, 0.27819997, ..., 0.07230002, 0.07230002,
       0.07230002])

In [43]:
dataset_df1 = dataset_df1.set_index('cas registry no')

In [44]:
dataset_df_labeled = labels_df.join(dataset_df1)
dataset_df_labeled

,Name,Formula,Mw,Elements,InChi,SMILES,alkane,methyl,alkene,alkyne,...,apodization,folding limits,number of interferograms averaged per single channel spectrum,$spectra version,$uncertainty in y,sample description,pressure,temperature,x1,y1
CAS,,,,,,,,,,,,,,,,,,,,,
100-00-5,"Benzene, 1-chloro-4-nitro-",C6H4ClNO2,156.993056,"['C', 'H', 'Cl', 'N', 'O']",InChI=1/C6H4ClNO2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H,[O-][N+](=O)c1ccc(Cl)cc1,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.026076922193169594, 0.02797572687268257, 0...."
100-01-6,p-Nitroaniline,C6H6N2O2,138.042927,"['C', 'H', 'N', 'O']",InChI=1/C6H6N2O2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H...,Nc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.005049999803304672, 0.005049999803304672, 0..."
100-02-7,"Phenol, 4-nitro-",C6H5NO3,139.026943,"['C', 'H', 'N', 'O']","InChI=1/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",Oc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0002783999952953309, 0.0002783999952953309,..."
1000-49-3,N-butyltrimethylsilane,C7H18Si,130.117777,"['C', 'H', 'Si']","InChI=1/C7H18Si/c1-5-6-7-8(2,3)4/h5-7H2,1-4H3",CCCC[Si](C)(C)C,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.08899998664855957, 0.08958601189397797, 0.0..."
1000-50-6,N-Butyldimethylchlorosilane,C6H15ClSi,150.063155,"['C', 'H', 'Cl', 'Si']","InChI=1/C6H15ClSi/c1-4-5-6-8(2,3)7/h4-6H2,1-3H3",CCCC[Si](C)(C)Cl,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.20800000429153442, 0.20800000429153442, 0.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99-97-8,"Benzenamine, N,N,4-trimethyl-",C9H13N,135.104799,"['C', 'H', 'N']","InChI=1/C9H13N/c1-8-4-6-9(7-5-8)10(2)3/h4-7H,1...",CN(C)c1ccc(C)cc1,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.02355931419879198, 0.02319779060781002, 0.0..."
999-78-0,"2-Pentyne, 4,4-dimethyl-",C7H12,96.093900,"['C', 'H']","InChI=1/C7H12/c1-5-6-7(2,3)4/h1-4H3",CC#CC(C)(C)C,1.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0013761000009253621, 0.0013761000009253621,..."
999-81-5,"Ammonium chloride, 2-chloroethyltrimethyl-",C5H13ClN.Cl,157.042505,"['C', 'H', 'Cl', 'N', 'Cl']","InChI=1/C5H13ClN.ClH/c1-7(2,3)5-4-6;/h4-5H2,1-...",[Cl-].C[N+](C)(C)CCCl,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.13220000267028809, 0.1312999725341797, 0.12..."


In [45]:
dataset_df_labeled.columns

Index(['Name', 'Formula', 'Mw', 'Elements', 'InChi', 'SMILES', 'alkane',
       'methyl', 'alkene', 'alkyne',
       ...
       'apodization', 'folding limits',
       'number of interferograms averaged per single channel spectrum',
       '$spectra version', '$uncertainty in y', 'sample description',
       'pressure', 'temperature', 'x1', 'y1'],
      dtype='object', length=101)

In [88]:
dataset_df_labeled1 = dataset_df_labeled[['Name', 'Formula', 'Mw', 'Elements', 'InChi', 'SMILES', 'alkane',
       'methyl', 'alkene', 'alkyne', 'alcohols', 'amines', 'nitriles',
       'aromatics', 'alkyl halides', 'esters', 'ketones', 'aldehydes',
       'carboxylic acids', 'ether', 'acyl halides', 'amides', 'nitro',
       'title',
       'molform',
       'x', 'y', 'x1', 'y1']]
dataset_df_labeled1

,Name,Formula,Mw,Elements,InChi,SMILES,alkane,methyl,alkene,alkyne,...,ether,acyl halides,amides,nitro,title,molform,x,y,x1,y1
CAS,,,,,,,,,,,,,,,,,,,,,
100-00-5,"Benzene, 1-chloro-4-nitro-",C6H4ClNO2,156.993056,"['C', 'H', 'Cl', 'N', 'O']",InChI=1/C6H4ClNO2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H,[O-][N+](=O)c1ccc(Cl)cc1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,"Benzene, 1-chloro-4-nitro-",C 6 H 4 Cl N O 2,"[450.0, 454.0, 458.0, 462.0, 466.0, 470.0, 474...","[0.025064226, 0.03873562, 0.039495144, 0.01962...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.026076922193169594, 0.02797572687268257, 0...."
100-01-6,p-Nitroaniline,C6H6N2O2,138.042927,"['C', 'H', 'N', 'O']",InChI=1/C6H6N2O2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H...,Nc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,p-Nitroaniline,C 6 H 6 N 2 O 2,"[551.688, 555.546, 559.404, 563.262, 567.12, 5...","[0.00505, 0.00536, 0.00556, 0.00599, 0.00634, ...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.005049999803304672, 0.005049999803304672, 0..."
100-02-7,"Phenol, 4-nitro-",C6H5NO3,139.026943,"['C', 'H', 'N', 'O']","InChI=1/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",Oc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,4-Nitrophenol,C 6 H 5 N O 3,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[0.0002784, 0.0001464, 0.0001752, 0.0002928, 0...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0002783999952953309, 0.0002783999952953309,..."
1000-49-3,N-butyltrimethylsilane,C7H18Si,130.117777,"['C', 'H', 'Si']","InChI=1/C7H18Si/c1-5-6-7-8(2,3)4/h5-7H2,1-4H3",CCCC[Si](C)(C)C,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,n-BUTYLTRIMETHYLSILANE,C7 H18 Si,"[203.607, 204.53778, 205.46855, 206.39932, 207...","[0.19099998, 0.19099998, 0.19099998, 0.1909999...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.08899998664855957, 0.08958601189397797, 0.0..."
1000-50-6,N-Butyldimethylchlorosilane,C6H15ClSi,150.063155,"['C', 'H', 'Cl', 'Si']","InChI=1/C6H15ClSi/c1-4-5-6-8(2,3)7/h4-6H2,1-3H3",CCCC[Si](C)(C)Cl,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,n-BUTYL CHLORODIMETHYL SILANE,C6 H15 Cl Si,"[625.2462, 625.5054, 625.7648, 626.0244, 626.2...","[0.208, 0.208, 0.20899999, 0.208, 0.208, 0.208...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.20800000429153442, 0.20800000429153442, 0.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99-97-8,"Benzenamine, N,N,4-trimethyl-",C9H13N,135.104799,"['C', 'H', 'N']","InChI=1/C9H13N/c1-8-4-6-9(7-5-8)10(2)3/h4-7H,1...",CN(C)c1ccc(C)cc1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,"Benzenamine, N,N,4-trimethyl-",C 9 H 13 N,"[450.0, 454.0, 458.0, 462.0, 466.0, 470.0, 474...","[0.006145908, 0.006989464, 0.008676576, 0.0113...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.02355931419879198, 0.02319779060781002, 0.0..."
999-78-0,"2-Pentyne, 4,4-dimethyl-",C7H12,96.093900,"['C', 'H']","InChI=1/C7H12/c1-5-6-7(2,3)4/h1-4H3",CC#CC(C)(C)C,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,"4,4-Dimethyl-2-pentyne",C 7 H 12,"[550.0, 554.0, 558.0, 562.0, 566.0, 570.0, 574...","[0.0013761, 0.0011286, 0.0008943, 0.0006864, 0...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0013761000009253621, 0.0013761000009253621,..."
999-81-5,"Ammonium chloride, 2-chloroethyltrimethyl-",C5H13ClN.Cl,157.042505,"['C', 'H', 'Cl', 'N', 'Cl']","InChI=1/C5H13ClN.ClH/c1-7(2,3)5-4-6;/h4-5H2,1-...",[Cl-].C[N+](C)(C)CCCl,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,"Ammonium chloride, 2-chloroethyltrimethyl-",C5 H13 Cl2 N,"[459.0, 460.0, 461.0, 462.0, 463.0, 464.0, 465...","[0.15679997, 0.15499997, 0.15319997, 0.1513000...","[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.13220000267028809, 0.1312999725341797, 0.12..."


In [66]:
spectra_x =dataset_df_labeled.iloc[:,-1]
spectra_x_t = torch.Tensor(spectra_x[0])
spectra_x_t

tensor([0.0261, 0.0280, 0.0325,  ..., 0.0071, 0.0071, 0.0071])

# Создание класса Dataset

Вивлеты - преобразование для детектирования пиков.

In [140]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

class IRSpectraDataset(Dataset):
    def __init__(self, dataset_df_labeled):
        self.dataset_df = dataset_df_labeled
        self.spectra_labels = self.dataset_df.iloc[:,6:23]
        self.spectra_x = self.dataset_df.iloc[:,-2]
        self.spectra_y = self.dataset_df.iloc[:,-1]
        
    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        
        spectra_x = torch.Tensor(self.spectra_x[idx]).reshape((1, 1800))
        spectra_y = torch.Tensor(self.spectra_y[idx]).reshape((1, 1800))      
        spectra = torch.cat(spectra_x, spectra_x)
        
        label = self.spectra_labels[idx]
        #transform
        return spectra, label

In [137]:
#spec = IRSpectraDataset()
x = dataset_df_labeled1.iloc[:,-2]
y = dataset_df_labeled1.iloc[:,-1]
spectra_x = torch.Tensor(x[0])
spectra_y = torch.Tensor(y[0])      
spectra = torch.cat((spectra_x.reshape((1, 1800)), spectra_y.reshape((1, 1800))), axis =0)
spectra

tensor([[5.0000e+02, 5.0200e+02, 5.0400e+02,  ..., 4.0940e+03, 4.0960e+03,
         4.0980e+03],
        [2.6077e-02, 2.7976e-02, 3.2533e-02,  ..., 7.0889e-03, 7.0889e-03,
         7.0889e-03]])

In [148]:
#Создание объекта класса и разбиение на трейн,тест и вал
from torch.utils.data import random_split
dataset_ir_spectra = IRSpectraDataset(dataset_df_labeled1)
train_size = int(0.7 * len(dataset_ir_spectra))
val_size = int(0.1 * len(dataset_ir_spectra))
test_size = len(dataset_ir_spectra) - train_size - val_size
train_dataset, test_dataset = random_split(dataset_ir_spectra, [train_size, int(test_size+val_size)])
test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

In [159]:
#Создание загрузчиков
# Number of classes
classes = list(dataset_ir_spectra.spectra_labels.columns)
num_classes = len(classes)
num_classes

17

In [ ]:
class_to_idx = #Work in progress

In [ ]:
# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset_ir_spectra.class_to_idx.items()}

# Код для обучения

Взят из https://github.com/Gan4x4/ml_snippets/blob/main/Training/Trainer.ipynb

In [ ]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_correct_count(pred, labels):
    _, predicted = torch.max(pred.data, 1)
    return (predicted.cpu() == labels.cpu()).sum().item()


@torch.inference_mode()  # this annotation disable grad computation
def validate(model, test_loader, device="cpu"):
    correct, total = 0, 0
    for imgs, labels in test_loader:
        pred = model(imgs.to(device))
        total += labels.size(0)
        correct += get_correct_count(pred, labels)
    return correct / total

In [ ]:
class Trainer:
    def __init__(self, model, plotter=None, lr=0.03):
        self.model = model
        self.optimizer = torch.optim.SGD(
            self.model.parameters(), lr=0.03
        )  # Weight update
        self.criterion = nn.CrossEntropyLoss()  # Loss function
        self.plotter = ProgressPlotter() if plotter is None else plotter
        self.epochs = 25
        self.loss_hist = []

    def __call__(self, train_loader, val_loader, epochs=10):
        global device
        print("Using device:", device)
        self.model.to(device)
        self.model.train()
        for epoch in tqdm(range(self.epochs)):
            self.loss_hist = []
            correct, total = 0, 0
            for imgs, labels in train_loader:
                correct += self.process_batch(imgs, labels)
                total += len(labels)
            self.plotter.add_scalar("Loss/train", np.mean(self.loss_hist))
            self.plotter.add_scalar(
                "Accuracy/val", validate(self.model, val_loader, device=device)
            )
            self.plotter.add_scalar("Accuracy/train", correct / total)
            self.plotter.display(["Loss/train", "Accuracy/val"])

    def process_batch(self, imgs, labels):
        self.optimizer.zero_grad()
        out = self.model(imgs.to(device))
        loss = self.criterion(out, labels.to(device))
        loss.backward()
        self.loss_hist.append(loss.item())
        self.optimizer.step()
        return get_correct_count(out.cpu(), labels)

Код для визуализации из прошлых заданий, можно заменить его собственым.

In [ ]:
from IPython.display import clear_output
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np


class ProgressPlotter:
    def __init__(self, title="default", groups=None) -> None:
        self._history_dict = defaultdict(dict)
        self.set_title(title)
        self.groups = self.get_groups(groups)

    def get_groups(self, groups):
        if groups is not None:
            return self._history_dict.keys()
        if type(groups) is str:
            groups = [groups]
        return groups

    def set_title(self, title):
        for g in self._history_dict.keys():
            self._history_dict[g][title] = []  # reset data
        self.title = title

    # group e.g. "loss_val" tag e.g. "experiment_1"
    def add_scalar(self, group: str, value, tag=None) -> None:
        tag = self.title if tag is None else tag

        if not tag in self._history_dict[group]:
            self._history_dict[group][tag] = []
        self._history_dict[group][tag].append(value)

    def add_row(self, group: str, value, tag=None) -> None:
        tag = self.title if tag is None else tag
        self._history_dict[group][tag] = value

    def display_keys(self, ax, data):
        history_len = 0
        ax.grid()
        for key in data:
            ax.plot(data[key], label=key)
            history_len = max(history_len, len(data[key]))
        if len(data) > 1:
            ax.legend(loc="upper right")
        if history_len < 50:
            ax.set_xlabel("step")
            ax.set_xticks(np.arange(history_len))
            ax.set_xticklabels(np.arange(history_len))

    """
     groups list of keys like [['loss_train','loss_val'],['accuracy']]
     All charts within a group will be plot in the same axis
  """

    def display(self, groups=None):
        clear_output()
        if groups is None:
            groups = self.groups
        n_groups = len(groups)
        fig, ax = plt.subplots(1, n_groups, figsize=(48 // n_groups, 3))
        if n_groups == 1:
            ax = [ax]
        for i, g in enumerate(groups):
            ax[i].set_ylabel(g)
            self.display_keys(ax[i], self.history_dict[g])
        fig.tight_layout()
        plt.show()

    @property
    def history_dict(self):
        return dict(self._history_dict)

# Модель

Предлагается использовать модель с несколькими свёрточными слоями(1) и понижением размерности до 225-450, а затем с полносвязыными слоями, где на выходе одна из 17 меток классов.

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
import torch.nn as nn
import torch

class TwoLayerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv1d(2, 8, 3), # in channel=2, out=32 3 - ядро свертки, таблица 1800 точек*2 столбца
            nn.MaxPool1d(2), # size [34,1800] -> [34,900] 
            nn.ReLU(),
            nn.Conv1d(8, 16, 3), # in channel=34, out=34
            nn.MaxPool1d(2), #  size [34,900] -> [34,450] 
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(7*7*32, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
            )

    def forward(self, x):
        # Your code here
        x = self.conv_stack(x) 
        return x

# Обучение модели

Оптимизатор - Адам с базовыми настройками
Лосс - Бинарная кросс-энтропия
Вывести табличку по классам sklearn пресижен реколл по классам
Какой-то репорт в sklearn 


# Оценка качества модели

# Выводы